## Web Scraping of Wikipedia Page Using Python

Here,I have scraped data from wikipedia page which consists of list of United States cities by population and created dataset using that.

#### Importing required Libraries

Imported all libraries required for web scrapping and for handling data

In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from urllib.request import urlopen

As required, I extracted data from Wikipedia webpage. The webpage includes a specific table with the names of cities of United States and their State, Area, Population, Location kind of details. And each city page contains all information for city. 

Defined function gettingHTMLData where I am passing URL of Wikipedia webpage and then using urlopen method. With the help of BeautifulSoup library, can get HTML content and can parse using html.parser

In [2]:
def gettingHTMLData(link):
    html = urlopen(link)
    soup = BeautifulSoup(html, "html.parser")
    return soup

Here, Using find_all() I am trying to fetch all the tables on the web page and applying table.prettify() to get content in more readable form.

In [3]:
content = gettingHTMLData("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")
tables = content.find_all("table")
for table in tables:
    print(table.prettify())

<table class="noprint infobox" id="GeoGroup" style="width: 23em; font-size: 88%; line-height: 1.5em">
 <tbody>
  <tr>
   <td>
    <b>
     Map all coordinates using:
    </b>
    <a class="external text" href="//tools.wmflabs.org/osm4wiki/cgi-bin/wiki/wiki-osm.pl?project=en&amp;article=List_of_United_States_cities_by_population">
     OpenStreetMap
    </a>
   </td>
  </tr>
  <tr>
   <td>
    <b>
     Download coordinates as:
    </b>
    <a class="external text" href="//tools.wmflabs.org/kmlexport?article=List_of_United_States_cities_by_population">
     KML
    </a>
    <b>
     ·
    </b>
    <a class="external text" href="http://tripgang.com/kml2gpx/http%3A%2F%2Ftools.wmflabs.org%2Fkmlexport%3Farticle%3DList_of_United_States_cities_by_population?gpx=1" rel="nofollow">
     GPX
    </a>
   </td>
  </tr>
 </tbody>
</table>

<table class="vertical-navbox nowraplinks navbox" style="float:right;clear:right;width:22.0em;margin:0 0 1.0em 1.0em;background:#f9f9f9;border:1px solid #aaa;padd

<table class="wikitable sortable" style="text-align:center">
 <tbody>
  <tr>
   <th>
    2018
    <br/>
    rank
   </th>
   <th>
    City
   </th>
   <th>
    State
    <sup class="reference" id="cite_ref-5">
     <a href="#cite_note-5">
      [c]
     </a>
    </sup>
   </th>
   <th>
    2018
    <br/>
    estimate
   </th>
   <th>
    2010
    <br/>
    Census
   </th>
   <th>
    Change
   </th>
   <th colspan="2">
    2016 land area
   </th>
   <th colspan="2">
    2016 population density
   </th>
   <th>
    Location
   </th>
  </tr>
  <tr>
   <td>
    1
   </td>
   <td style="text-align:left;background-color:#cfecec">
    <i>
     <a href="/wiki/New_York_City" title="New York City">
      New York City
     </a>
    </i>
    <sup class="reference" id="cite_ref-6">
     <a href="#cite_note-6">
      [d]
     </a>
    </sup>
   </td>
   <td align="left">
    <span class="flagicon">
     <img alt="" class="thumbborder" data-file-height="450" data-file-width="900" decoding="async" h

I figured out that, we need table which has class :'wikitable sortable'
List of all cities and Information about each of them is stored and each city name is basically a hyperlink for the specific webpage of city.



2nd row has all the links of cities so checking all 'td' tag in table rows and fetching links from cells[1]

In [4]:
table = content.find("table", {"class": "wikitable sortable"})
rows = table.find_all("tr")

# All city links
for row in rows:
    cells = row.find_all("td")
    if len(cells)>1:
        country_link = cells[1].find("a")
        print(country_link.get("href"))

/wiki/New_York_City
/wiki/Los_Angeles
/wiki/Chicago
/wiki/Houston
/wiki/Phoenix,_Arizona
/wiki/Philadelphia
/wiki/San_Antonio
/wiki/San_Diego
/wiki/Dallas
/wiki/San_Jose,_California
/wiki/Austin,_Texas
/wiki/Jacksonville,_Florida
/wiki/Fort_Worth,_Texas
/wiki/Columbus,_Ohio
/wiki/San_Francisco
/wiki/Charlotte,_North_Carolina
/wiki/Indianapolis
/wiki/Seattle
/wiki/Denver
/wiki/Washington,_D.C.
/wiki/Boston
/wiki/El_Paso,_Texas
/wiki/Detroit
/wiki/Nashville,_Tennessee
/wiki/Portland,_Oregon
/wiki/Memphis,_Tennessee
/wiki/Oklahoma_City
/wiki/Las_Vegas
/wiki/Louisville,_Kentucky
/wiki/Baltimore
/wiki/Milwaukee
/wiki/Albuquerque,_New_Mexico
/wiki/Tucson,_Arizona
/wiki/Fresno,_California
/wiki/Mesa,_Arizona
/wiki/Sacramento,_California
/wiki/Atlanta
/wiki/Kansas_City,_Missouri
/wiki/Colorado_Springs,_Colorado
/wiki/Miami
/wiki/Raleigh,_North_Carolina
/wiki/Omaha,_Nebraska
/wiki/Long_Beach,_California
/wiki/Virginia_Beach,_Virginia
/wiki/Oakland,_California
/wiki/Minneapolis
/wiki/Tulsa,_Okla

Here Each row has a link to the corresponding city page on Wikipedia but base link is not there so, we need to append that link.

After getting full URL of each city page, I figured out that table: 'infobox geography vcard' has details mentioned. Therefore, extracted Text of 'Time zone' and link of 'Website' from each webpage.

I defined the variable specific_details to collect all the information from each webpage in an array which we can then append with the list of cities dataset.

All the headings have class 'mergedtoprow' so using that we can get access of all the 'th' tags and can append table data to our list

In [5]:
def getSpecificDetails(url):
    try:
        specific_details=[]
        country_page = gettingHTMLData('https://en.wikipedia.org' + url)
        table = country_page.find('table', {'class': 'infobox geography vcard'})

        for tr in table.find_all('tr'):
            if(tr.get('class')==['mergedtoprow']):
                for th in tr.find_all('th'):
                    if(th.getText().strip()!='' and th.getText().strip()=='Time zone'):
                        specific_details.append(tr.find('td').get_text().strip('\n'))
                    if(th.getText().strip()!='' and th.getText().strip()=='Website'):
                        specific_details.append(tr.find('td').find('a').get('href'))
                        
        return specific_details
    except Exception as error:
        print('Error occured: {}'.format(error))
        return []    

### Creating Dataset

Firstly I read each row of the table from the list of cities and gather data from the first given page. After that,  I used the link to get specific details like Time zone and website link of each city.

Compiled data_content in dataframe using pandas.

In [6]:
data_content = []

for row in rows:
    cells = row.find_all('td')
    if len(cells) > 1:
        print(cells[1].get_text())
        country_link = cells[1].find('a')
        country_info = [cell.text.strip('\n') for cell in cells]
        specific_details = getSpecificDetails(country_link.get('href'))
        country_info += specific_details
        data_content.append(country_info)

dataset = pd.DataFrame(data_content)

New York City[d]

Los Angeles

Chicago

Houston[3]

Phoenix

Philadelphia[e]

San Antonio

San Diego

Dallas

San Jose

Austin

Jacksonville[f]

Fort Worth

Columbus

San Francisco[g]

Charlotte

Indianapolis[h]

Seattle

Denver[i]

Washington, D.C.[j]

Boston

El Paso

Detroit

Nashville[k]

Portland

Memphis

Oklahoma City

Las Vegas

Louisville[l]

Baltimore[m]

Milwaukee

Albuquerque

Tucson

Fresno

Mesa

Sacramento

Atlanta

Kansas City

Colorado Springs

Miami

Raleigh

Omaha

Long Beach

Virginia Beach[m]

Oakland

Minneapolis

Tulsa

Arlington

Tampa

New Orleans[n]

Wichita

Cleveland

Bakersfield

Aurora

Anaheim

Honolulu[b]

Santa Ana

Riverside

Corpus Christi

Lexington[o]

Stockton

Henderson

Saint Paul

St. Louis[m]

Cincinnati

Pittsburgh

Greensboro

Anchorage[p]

Plano

Lincoln

Orlando

Irvine

Newark

Toledo

Durham

Chula Vista

Fort Wayne

Jersey City

St. Petersburg

Laredo

Madison

Chandler

Buffalo

Lubbock

Scottsdale

Reno

Glendale

Gilbert[q]

Winston–S

Defined headers for the dataset

In [9]:
headers=['2018 rank','City','State','2018 estimate','2010 census','Change','2016 land area sq mi','2016 land area km2','2016 population density per sq mi','2016 population density per km2','Location','Time zone','Website']
dataset.columns = headers

Using sample, can see random samples from whole dataset.

In [10]:
dataset.sample(4)

,2018 rank,City,State,2018 estimate,2010 census,Change,2016 land area sq mi,2016 land area km2,2016 population density per sq mi,2016 population density per km2,Location,Time zone,Website
251,252,Temecula,California,"114,742","100,097",+14.63%,37.3 sq mi,96.6 km2,"3,031/sq mi","1,170/km2",33°29′35″N 117°07′54″W﻿ / ﻿33.4931°N 117.1317°...,UTC−8 (Pacific),http://temeculaca.gov
257,258,Miami Gardens,Florida,"113,069","107,167",+5.51%,18.2 sq mi,47.1 km2,"6,212/sq mi","2,398/km2",25°56′56″N 80°14′37″W﻿ / ﻿25.9489°N 80.2436°W﻿...,UTC-5 (EST),http://www.miamigardens-fl.gov/
90,91,Norfolk[m],Virginia,"244,076","242,803",+0.52%,53.3 sq mi,138.0 km2,"4,599/sq mi","1,776/km2",36°55′23″N 76°14′41″W﻿ / ﻿36.9230°N 76.2446°W﻿...,UTC−5 (EST),http://www.norfolk.gov/
197,198,Surprise,Arizona,"138,161","117,517",+17.57%,107.9 sq mi,279.5 km2,"1,230/sq mi",470/km2,33°40′14″N 112°27′10″W﻿ / ﻿33.6706°N 112.4527°...,UTC-7 (MST (no DST)),http://www.surpriseaz.gov


Here we can see, 2016 land area and population density is in sq mi as well km2 , so dropping column which consists of sq mi values and keeping only km2 values 

In [11]:
drop_columns = ['2016 land area sq mi', '2016 population density per sq mi']
dataset.drop(drop_columns, axis = 1, inplace = True)

Storing dataset in csv file

In [12]:
dataset.to_csv("Data1.csv", index = False)

In next cleanData.ipynb file, I have done cleaning of this dataset to upload on BigQuery table